In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [10]:
if is_jupyter: 
    seed        = 43
    num_patients = 20
    num_providers = 20
    provider_capacity = 1
    noise = 0.1
    fairness_constraint = -1
    num_trials = 100
    utility_function = "semi_synthetic_comorbidity"
    order="uniform"
    online_arrival = True  
    new_provider = False 
    out_folder = "dynamic"
    average_distance = 20.2
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=1)
    parser.add_argument('--noise', help='Noise in theta', type=float, default=0.1)
    parser.add_argument('--average_distance', help='Maximum distance patients are willing to go', type=float, default=20.2)
    parser.add_argument('--fairness_constraint', help='Maximum difference in average utility between groups', type=float, default=-1)
    parser.add_argument('--num_trials', help='Number of trials', type=int, default=100)
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument("--online_arrival",action="store_true",help="Patients arrive one-by-one")
    parser.add_argument("--new_provider",action="store_true",help="Are we simulating a new provider matching")
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    num_trials = args.num_trials
    noise = args.noise
    average_distance = args.average_distance
    fairness_constraint = args.fairness_constraint
    provider_capacity = args.provider_capacity
    utility_function = args.utility_function
    order = args.order
    online_arrival = args.online_arrival
    new_provider = args.new_provider 
    out_folder = args.out_folder
    
assert not(online_arrival and new_provider)
save_name = secrets.token_hex(4)  

In [11]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'utility_function': utility_function, 
        'order': order, 
        'num_trials': num_trials, 
        'noise': noise, 
        'average_distance': average_distance,
        'online_arrival': online_arrival,
        'new_provider': new_provider,
        'fairness_constraint': fairness_constraint} 

## Baselines

In [12]:
seed_list = [seed]
restrict_resources()

In [23]:
policy = one_shot_policy
per_epoch_function = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['random_num_matches'])/num_patients,np.mean(results['random_patient_utilities'])))

random policy
Matches 0.35113333333333335, Utilities 0.4230331755178863


In [22]:
policy = one_shot_policy
per_epoch_function = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy policy
Matches 0.38089999999999996, Utilities 0.4385465585590219


In [21]:
policy = one_shot_policy
if fairness_constraint != -1:
    per_epoch_function = get_fair_optimal_policy(fairness_constraint,seed)
else:
    per_epoch_function = optimal_policy
name = "omniscient_optimal"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

omniscient_optimal policy
Matches 0.4033333333333333, Utilities 0.45082582779808955


## Optimization-Based

In [12]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

lp policy
Set parameter Username


Set parameter LicenseID to value 2709943
Academic license - for non-commercial use only - expires 2026-09-17
Matches 0.4, Utilities 0.44811056957441925


In [15]:
policy = one_shot_policy
if fairness_constraint != -1:
    per_epoch_function = get_gradient_policy(num_patients,num_providers,fairness_constraint,seed)
else:
    per_epoch_function = gradient_policy
name = "gradient_descent"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

gradient_descent policy
On trial 0
On trial 1
On trial 2
On trial 3
On trial 4
On trial 5
On trial 6
On trial 7
On trial 8
On trial 9
On trial 10
On trial 11
On trial 12
On trial 13
On trial 14
On trial 15
On trial 16
On trial 17
On trial 18
On trial 19
On trial 20
On trial 21
On trial 22
On trial 23
On trial 24
On trial 25
On trial 26
On trial 27
On trial 28
On trial 29
On trial 30
On trial 31
On trial 32
On trial 33
On trial 34
On trial 35
On trial 36
On trial 37
On trial 38
On trial 39
On trial 40
On trial 41
On trial 42
On trial 43
On trial 44
On trial 45
On trial 46
On trial 47
On trial 48
On trial 49
On trial 50
On trial 51
On trial 52
On trial 53
On trial 54
On trial 55
On trial 56
On trial 57
On trial 58
On trial 59
On trial 60
On trial 61
On trial 62
On trial 63
On trial 64
On trial 65
On trial 66
On trial 67
On trial 68
On trial 69
On trial 70
On trial 71
On trial 72
On trial 73
On trial 74
On trial 75
On trial 76
On trial 77
On trial 78
On trial 79
On trial 80
On trial 81
On

KeyboardInterrupt: 

## Save Data

In [81]:
save_path = get_save_path(out_folder,save_name)

In [82]:
delete_duplicate_results(out_folder,"",results)

In [83]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)